### Importação das Bibliotecas

In [7]:
import numpy as np
import cv2
import glob
import time
import matplotlib.image as mpimg
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip

## Obtenção do dados e analise dos dados para treinamento

In [3]:
# Retorna dois vetores com o caminho e nome dos arquivos das imagens que serão processadas.
def pathimage(vehicle, non_vehicle, extension='png'):
    vehicle_image_filenames = glob.glob(vehicle + '**/*.' + extension) #, recursive=True)
    non_vehicle_image_filenames = glob.glob(non_vehicle + '**/*.' + extension)  #, recursive=True)
    return vehicle_image_filenames, non_vehicle_image_filenames

In [6]:
# Retorna propriedades HOG e visualizacao
def gethogfeatures(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True, L = 'L2-Hys'):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), 
                                  cells_per_block=(cell_per_block, cell_per_block), # block_norm=L,
                                  transform_sqrt=True, visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), # block_norm=L,
                       transform_sqrt=True, visualise=vis, feature_vector=feature_vec)
        return features

## Metodo de extração de propriedades

In [4]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=2):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else:
            feature_image = np.copy(image)

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(gethogfeatures(img=feature_image[:, :, channel], orient=orient,
                                                      pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                                      vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)
        else:
            hog_features = gethogfeatures(img=feature_image[:, :, hog_channel], orient=orient,
                                             pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, vis=False,
                                             feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features

## Treinamento do Modelo

In [5]:
def train_model(vehicle_hog_features, non_vehicle_hog_features):
    parameters = [
        {
            # 'C': [0.01],
            'C': [1e-2, 1, 1e2],
            # Cost - Penalização de elementos dentro da margem de suporte
            'kernel': ['linear']
            # Kernel - Define a função de custo que será minimizada
        }
    ]

    f1_scorer = make_scorer(f1_score)

    grid_obj = GridSearchCV(SVC(), param_grid=parameters, scoring=f1_scorer)


    # Create an array stack of feature vectors
    X = np.vstack((vehicle_hog_features, non_vehicle_hog_features)).astype(np.float64)

    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)

    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    # Defini o vetor Y com a label
    y = np.hstack((np.ones(len(vehicle_hog_features)), np.zeros(len(non_vehicle_hog_features))))

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)

    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state, stratify=y)

    print('Using:', orient, 'orientations', pix_per_cell, 'pixels per cell and', cell_per_block, 'cells per block')
    print('Training data set size: ', len(X_train))
    print('Testing data set size: ', len(X_test))

    # Check the training time for the SVC
    t = time.time()
    grid_obj.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2 - t, 2), 'Seconds to train SVC...')

    print(grid_obj.best_params_)

    clf = grid_obj.best_estimator_

    # Check the score of the SVC

    # Check the prediction time for a single sample
    t = time.time()

    y_preds = clf.predict(X_test)
    f1 = f1_score(y_test, y_preds)

    t2 = time.time()

    print(round(t2 - t, 5), 'Seconds to Predict')
    print(f1)

    return clf, X_scaler

In [ ]:
'''
Execution
'''

pix_per_cell = 8
cell_per_block = 2
orient = 10
color_space = 'YCrCb'
hog_channel = 'ALL'

vehicle_image_filenames, non_vehicle_image_filenames = pathimage('training_images/vehicles/', 'training_images/non-vehicles/')
print('Arquivos contendo veiculos: ', len(vehicle_image_filenames))
print('Arquivos sem veiculos: ', len(non_vehicle_image_filenames))

print('Fez a leitura dos nomes dos arquivos...')

t1 = time.time()
vehicle_hog_features = extract_features(vehicle_image_filenames, color_space, orient, pix_per_cell, cell_per_block, hog_channel)

non_vehicle_hog_features = extract_features(non_vehicle_image_filenames, color_space, orient, pix_per_cell, cell_per_block, hog_channel)
t2 = time.time()
print(round(t2-t1, 2), ' para fazer os HOGs...')

model, scaler = train_model(vehicle_hog_features, non_vehicle_hog_features)
print('Fez o treinamento do modelo...')

paramhog = {
    'pix_per_cell': pix_per_cell,
    'cell_per_block': cell_per_block,
    'orient': orient,
    'color_space': color_space,
    'hog_channel': hog_channel
}

#salva o modelo treinado
joblib.dump(model, 'model/vehiclemodel.pkl')
print('Salvou o modelo...')

joblib.dump(scaler, 'model/vehiclescaler.pkl')
print('Salvou a escala...')

joblib.dump(paramhog, 'model/paramhog.pkl')
print('Salvou os parametros do Hog...')

print('Finalizou !!!')